In [1]:
%reload_ext autoreload
%autoreload 2

In [11]:
import pandas as pd

from depsurf import DepKind, DiffResult, IssueEnum, VersionGroup
from utils import load_pkl, GRAY_DASH, save_latex

data: DiffResult = load_pkl("config")

ISSUES = {
    IssueEnum.ADD: "$+$",
    IssueEnum.REMOVE: "$-$",
    IssueEnum.CHANGE: "$\Delta$",
    IssueEnum.NEW: "#",
}

KINDS = {
    DepKind.STRUCT: "Struct",
    DepKind.CONFIG: "Config",
    DepKind.FUNC: "Func",
    DepKind.TRACEPOINT: r"\shortstack{Tracept\\($\Delta=0$)}",
    DepKind.SYSCALL: r"Syscall",
}


def format_val(v):
    if v > 1000:
        return f"{v/1000:.1f}k"
    return v


results = {}
for i, ((g, p), d) in enumerate(data.items()):
    if p.v2.flavor == "oracle":
        continue

    if i == 0:
        result = {}
        for (kind, s), v in d.items():
            if s == IssueEnum.OLD:
                result[(KINDS[kind], ISSUES[IssueEnum.NEW])] = format_val(v)
            elif (
                s in (IssueEnum.ADD, IssueEnum.REMOVE, IssueEnum.CHANGE)
                and kind != DepKind.CONFIG
            ):
                result[(KINDS[kind], ISSUES[s])] = 0
        result[("Register", ISSUES[IssueEnum.CHANGE])] = 0
        results[(r"\multicolumn{1}{c|}{def-}", r"\multicolumn{1}{c|}{ault}")] = result

    result = {}
    for (kind, s), v in d.items():
        if kind == DepKind.CONFIG and s != IssueEnum.NEW:
            continue
        kind = KINDS[kind]
        s = ISSUES.get(s)
        if not s:
            continue
        result[(kind, s)] = format_val(v)
    result[("Register", ISSUES[IssueEnum.CHANGE])] = 1 if g == VersionGroup.ARCH else 0
    name = g.to_str(p.v2)
    # if name == "low-lat":
    #     name = "lo lat"
    if g == VersionGroup.ARCH:
        g = "Architecture"
        name = r"\texttt{\footnotesize " + name + r"}"
    results[(g, name)] = result


df = pd.DataFrame(results)
# df = df.loc[(df != 0).any(axis=1) | (df.index.get_level_values(1) == "$\Delta$")]
df = df.loc[(df != 0).any(axis=1)]


df_latex = df.map(lambda x: GRAY_DASH if x == 0 else x)
latex = df_latex.to_latex(
    multicolumn_format="c|", column_format=r"p{7ex}p{1.5ex}|r|rrrr|rrrr", multirow=True
)

save_latex(latex, "cfg", rotate=False)
df

[ utils_pickle.py:18 ] INFO: Loding config from /Users/szhong/Downloads/bpf-study/output/config.pkl
[  utils_latex.py:69 ] INFO: Saved cfg to /Users/szhong/Downloads/bpf-study/paper/tabs/cfg.tex


\multicolumn{1}{c|}{def-}                 Architecture                                                                                      Flavor                     
                                            \multicolumn{1}{c|}{ault} \texttt{\footnotesize arm64} \texttt{\footnotesize arm32} \texttt{\footnotesize ppc} \texttt{\footnotesize s390} low-lat    AWS  Azure    GCP
Config                             #                             8.8k                         9.6k                         9.6k                       8.1k                        3.0k    8.8k   6.4k   5.3k   8.6k
Func                               #                            48.0k                        49.4k                        48.8k                      42.8k                       33.2k   48.0k  46.5k  45.5k  48.1k
                                   $+$                              0                         9.2k                        12.6k                       5.4k                        3.9k      57    328    992    450
                                   $-$                              0                         7.9k                        11.8k                      10.6k                       18.8k      41   1.8k   3.5k    319
                                   $\Delta$                         0                          120                          106                        137                          78       0      2     10      1
Struct                             #                             8.4k                         9.1k                         8.6k                       7.4k                        6.1k    8.4k   8.0k   7.8k   8.4k
                                   $+$                              0                         1.7k                         2.0k                        570                         694       4     83    257     68
                                   $-$                              0                         1.0k                         1.9k                       1.6k                        3.1k       1    483    833    123
                                   $\Delta$                         0                           81                          154                        116                         115       5     19     28     14
\shortstack{Tracept\\($\Delta=0$)} #                              752                          685                          690                        648                         559     752    747    739    752
                                   $+$                              0                           45                           70                         25                          61       0      4     26      0
                                   $-$                              0                          112                          132                        129                         254       0      9     39      0
Syscall                            #                              333                          291                          378                        347                         341     333    333    333    333
                                   $+$                              0                            2                           74                         23                          22       0      0      0      0
                                   $-$                              0                           44                           29                          9                          14       0      0      0      0
Register                           $\Delta$                         0                            1                            1                          1                           1       0      0      0      0